In [ ]:
import psycopg2
import pandas as pd
import os
import itertools
import numpy as np

conn = psycopg2.connect(
    host='localhost',
    port=54320,
    dbname='my_database',
    password='pass',
    user='user',
    )
conn.autocommit = False


In [ ]:
df = pd.read_csv("FeatureTableCSV.csv", sep=";")

In [ ]:
df

In [ ]:
df.where(df["NazwaFolderu"] == "FrequencyOfPosts")

In [ ]:
list(df.loc[df["NazwaFolderu"] == "FrequencyOfPosts"]["NazwaCechy"])

In [ ]:
import datetime
from datetime import timedelta 

authors = "authors"
comments = "comments"
posts = "posts"
category = "category"
topics = "topics"
user_id = 590

start_date_comments = datetime.date(2008, 12, 9)
end_date_comments = datetime.date(2013,11, 16)
# start_date_comments = datetime.date(2011, 12, 24)
# end_date_comments = datetime.date(2012, 2, 23)
# 2008-12-09 18:41:00 - comments 
# 2013-11-16 21:35:00 - comments

# 2005-12-09 09:50:17 - posts
# 2013-11-14 10:13:00 - posts


In [ ]:
def get_all_table(table_name):
    cur = conn.cursor()
    query = """
    SELECT * FROM {}
    """.format(table_name)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
def get_all_users():
    cur = conn.cursor()
    query = """
    SELECT id, name FROM {} order by id
    """.format(authors)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
def get_all_ids():
    result = get_all_users()
    return [x[0] for x in result]

In [ ]:
len(get_all_ids())

In [ ]:
def save_data_to_file(folder_name, file_name, data):
    try:
        # Create target Directory
        os.mkdir(folder_name)
        print("Directory " , folder_name ,  " Created ") 
    except FileExistsError:
        e = 1
        #print("Directory " , folder_name ,  " already exists")
    data.to_csv(folder_name + "/" + file_name, index=False)

In [ ]:
#########


# number of written comments to others users - months periods
def get_number_of_written_comments_to_other_users(user_id, start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT COUNT(*) FROM {} c
    JOIN {} p ON p.id = c.post_id
    WHERE c.author_id = {} and p.author_id <> {} AND
    c.date between '{}' and '{}'
    """.format(comments, posts, user_id, user_id, start_date, end_date)
    cur.execute(query)
    res = cur.fetchone()
    conn.commit()
    cur.close()
    return res
#     conn.close()


In [ ]:
# res = get_number_of_written_comments_to_other_users(591, start_date_comments, start_date_comments + timedelta(days=30))
def generate_number_of_written_comments_to_other_users(users):
    start_date = start_date_comments
    while start_date < end_date_comments:
        print(start_date)
        data = []
        for user_id in users:
            written_com = get_number_of_written_comments_to_other_users(user_id, start_date, start_date + timedelta(days=30))
            result = []
            result.append(written_com[0])
            result.append(user_id)
            result.append(str(start_date))
            data.append(result)
        df = pd.DataFrame(data, columns = ['number_of_written_comments_to_other_users', 'user_id', 'start_date'])
        save_data_to_file("WrittenCommToOthers", "feateures_" + str(start_date) + ".csv", df)
        start_date += timedelta(days=15)
        

In [ ]:
################

# the number of received responses (comments) for user’s posts
def get_number_of_received_responses_for_users_posts(user_id, start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT AVG(cs.resp_amount), stddev(cs.resp_amount),
    percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.resp_amount),
    percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.resp_amount),
    MAX(cs.resp_amount), MIN(cs.resp_amount) FROM
    (SELECT COUNT(*) as resp_amount FROM {} c
    JOIN {} p ON p.id = c.post_id
    WHERE p.author_id = {} and c.author_id <> {} AND
    c.date between '{}' and '{}'
    GROUP BY p.id) as cs
    """.format(comments, posts, user_id, user_id, start_date, end_date)
    cur.execute(query)
    res = cur.fetchone()
    conn.commit()
    cur.close()
    return res

In [ ]:
def generate_number_of_received_responses_for_users_posts(users):
    start_date = start_date_comments
    while start_date < end_date_comments:
        print(start_date)
        data = []
        for user_id in users:
            receive_posts = get_number_of_received_responses_for_users_posts(user_id, start_date, start_date + timedelta(days=30))
            #print("Proceding data: of user: {}".format(user_id))
            receive_posts = [0 if v is None else v for v in receive_posts]
            #print(receive_posts)
            result = []
            result.extend(receive_posts)
            result.append(user_id)
            result.append(str(start_date))
            data.append(result)
        
        df = pd.DataFrame(data, columns = ['avg_responses_to_posts','std_responses', 'median_responses_to_posts','q3','min','max', 'user_id', 'start_date'])
        save_data_to_file("NumOfReceiveResponseToUserPosts", "feateures_" + str(start_date) + ".csv", df)
        start_date += timedelta(days=15)
        

In [ ]:
# the number of comments written by user under his own posts


##############################

def get_number_of_comments_written_by_user_under_his_own_posts(user_id, start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT AVG(cs.resp_amount), stddev(cs.resp_amount),
    percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.resp_amount),
    percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.resp_amount),
    MAX(cs.resp_amount), MIN(cs.resp_amount) FROM
    (SELECT COUNT(*) as resp_amount FROM {} c
    JOIN {} p ON p.id = c.post_id
    WHERE p.author_id = {} and c.author_id = {} AND
    c.date between '{}' and '{}'
    GROUP BY p.id) as cs
    """.format(comments, posts, user_id, user_id, start_date, end_date)
    cur.execute(query)
    res = cur.fetchone()
    conn.commit()
    cur.close()
    return res

In [ ]:
def generate_number_of_comments_written_by_user_under_his_own_posts(users):
    start_date = start_date_comments
    while start_date < end_date_comments:
        print(start_date)
        data = []
        for user_id in users:
            comm = get_number_of_comments_written_by_user_under_his_own_posts(user_id, start_date, start_date + timedelta(days=30))
            #print("Proceding data: of user: {}".format(user_id))
            comm = [0 if v is None else v for v in comm]
            #print(receive_posts)
            result = []
            result.extend(comm)
            result.append(user_id)
            result.append(str(start_date))
            data.append(result)
        
        df = pd.DataFrame(data, columns = ['avg_comments_to_own_posts','std_comments', 'median_responses_to_posts','q3','min','max', 'user_id', 'start_date'])
        save_data_to_file("NumOfOwnCommToUserPosts", "feateures_" + str(start_date) + ".csv", df)
        start_date += timedelta(days=15)

In [ ]:
# the number of received responses under user’s comments

######################################

def get_number_of_received_responses_under_users_comments(user_id, start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT AVG(cs.resp_amount), stddev(cs.resp_amount),
    percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.resp_amount),
    percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.resp_amount),
    MAX(cs.resp_amount), MIN(cs.resp_amount) FROM
    (SELECT COUNT(*) as resp_amount FROM {} c
    JOIN {} c2 ON c.parentcomment_id = c2.id
    WHERE c2.author_id = {} and c.author_id <> {} AND
    c.date between '{}' and '{}'
    GROUP BY c.parentcomment_id) as cs
    """.format(comments, comments, user_id, user_id, start_date, end_date)
    cur.execute(query)
    res = cur.fetchone()
    conn.commit()
    cur.close()
    return res

In [ ]:
def generate_number_of_received_responses_under_users_comments(users):
    start_date = start_date_comments
    while start_date < end_date_comments:
        print(start_date)
        data = []
        for user_id in users:
            comm = get_number_of_received_responses_under_users_comments(user_id, start_date, start_date + timedelta(days=30))
            #print("Proceding data: of user: {}".format(user_id))
            comm = [0 if v is None else v for v in comm]
            #print(receive_posts)
            result = []
            result.extend(comm)
            result.append(user_id)
            result.append(str(start_date))
            data.append(result)
        
        df = pd.DataFrame(data, columns = ['avg_comments_to_comm','std_comments', 'median_responses_to_comm','q3','min','max', 'user_id', 'start_date'])
        save_data_to_file("NumOfCommToUserComm", "feateures_" + str(start_date) + ".csv", df)
        start_date += timedelta(days=15)

In [ ]:
#####################

# number of user’s posts
def get_number_of_users_posts(user_id, start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT COUNT(*) FROM {} WHERE author_id = {} AND
    date between '{}' and '{}'
    """.format(posts, user_id, start_date, end_date)
    cur.execute(query)
    res = cur.fetchone()
    conn.commit()
    cur.close()
    return res

In [ ]:
def generate_number_of_users_posts(users):
    start_date = start_date_comments
    while start_date < end_date_comments:
        print(start_date)
        data = []
        for user_id in users:
            posts = get_number_of_users_posts(user_id, start_date, start_date + timedelta(days=30))
            #print("Proceding data: of user: {}".format(user_id))
            posts = [0 if v is None else v for v in posts]
            #print(receive_posts)
            result = []
            result.extend(posts)
            result.append(user_id)
            result.append(str(start_date))
            data.append(result)
        
        df = pd.DataFrame(data, columns = ['num_of_posts', 'user_id', 'start_date'])
        save_data_to_file("NumOfUsersPosts", "feateures_" + str(start_date) + ".csv", df)
        start_date += timedelta(days=15)

In [ ]:
def get_freq_stats(dates, timedelta_days):
    dates = [x for x in dates if x is not None]
    dates = sorted(dates)
    freq_stats = []
    i = 0
    d = dates[0]
    while i < len(dates):
        posts = 0
        while i < len(dates):
            if dates[i] < d + timedelta(days=timedelta_days):
                posts += 1
            else:
                break
            i += 1
        freq_stats.append(posts)
        d += timedelta(days=timedelta_days)
    freq_stats = np.array(freq_stats)
    activity_time = dates[len(dates) - 1] - dates[0]
    return (freq_stats, activity_time)

In [ ]:
def calculate_posts_frequency(dates, timedelta_days):
    posts_stats, activity_time = get_freq_stats(dates, timedelta_days)
    return {
        'posts_activity_time': activity_time.days + 1,
        'posts_freq_avg': round(np.average(posts_stats),2),
        'posts_freq_stddev': round(np.std(posts_stats), 2),
        'posts_freq_median': round(np.median(posts_stats), 2),
        'posts_freq_q3': round(np.percentile(posts_stats, 75),2),
        'posts_freq_max': np.max(posts_stats),
        'posts_freq_min': np.min(posts_stats)
    }

In [ ]:
def calculate_comments_frequency(dates, timedelta_days):
    comm_stats, activity_time = get_freq_stats(dates, timedelta_days)
    return {
        'comments_freq_avg': round(np.average(comm_stats),2),
        'comments_freq_stddev': round(np.std(comm_stats), 2),
        'comments_freq_median': round(np.median(comm_stats), 2),
        'comments_freq_q3': round(np.percentile(comm_stats, 75),2),
        'comments_freq_max': np.max(comm_stats),
        'comments_freq_min': np.min(comm_stats)
    }

In [ ]:
# time period in which user was actively writing posts - jak to liczyczy w ciagu miesiaca ?
# post_activity time + frequency of writing posts
def get_post_activity_time_and_frequency(user_id, start_date, end_date, timedelta_days=7):
    cur = conn.cursor()
    query = """
    SELECT date FROM {} WHERE author_id = {} AND
    date between '{}' and '{}'
    """.format(posts, user_id, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    dates = []
    for i in range(len(res)):
        dates.append(res[i][0])
    if len(res) > 0:
        return calculate_posts_frequency(dates, timedelta_days)
    else:
        return {
            'posts_activity_time': 0.0,
            'posts_freq_avg': 0.0,
            'posts_freq_stddev': 0.0,
            'posts_freq_median': 0.0,
            'posts_freq_q3': 0.0,
            'posts_freq_max': 0.0,
            'posts_freq_min': 0.0
            }

In [ ]:
def generate_post_activity_time_and_frequency(users):
    start_date = start_date_comments
    while start_date < end_date_comments:
        print(start_date)
        data = []
        for user_id in users:
            posts = get_post_activity_time_and_frequency(user_id, start_date, start_date + timedelta(days=30)).values()
            #print("Proceding data: of user: {}".format(user_id))
            posts = [0 if v is None else v for v in posts]
            #print(receive_posts)
            result = []
            result.extend(posts)
            result.append(user_id)
            result.append(str(start_date))
            data.append(result)
        
        df = pd.DataFrame(data, columns = ['posts_activity_time', 'posts_freq_avg', 'posts_freq_stddev', 'posts_freq_median', 'posts_freq_q3', 'posts_freq_max', 'posts_freq_min', 'user_id', 'start_date'])
        save_data_to_file("NumOfUsersPostsFrequency", "feateures_" + str(start_date) + ".csv", df)
        start_date += timedelta(days=15)

In [ ]:
# frequency of writting comments
def get_comments_frequency(user_id, start_date, end_date, timedelta_days=7):
    cur = conn.cursor()
    query = """
    SELECT date FROM {} WHERE author_id = {} AND
    date between '{}' and '{}'
    """.format(comments, user_id, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    dates = []
    for i in range(len(res)):
        dates.append(res[i][0])
    if len(res) > 0:
        return calculate_comments_frequency(dates, timedelta_days)
    else:
        return {
                'comments_freq_avg': 0.0,
                'comments_freq_stddev': 0.0,
                'comments_freq_median': 0.0,
                'comments_freq_q3': 0.0,
                'comments_freq_max': 0.0,
                'comments_freq_min': 0.0
            }

In [ ]:
def generate_comments_frequency(users):
    start_date = start_date_comments
    while start_date < end_date_comments:
        print(start_date)
        data = []
        for user_id in users:
            posts = get_comments_frequency(user_id, start_date, start_date + timedelta(days=30)).values()
            #print(posts.values())
            #print("Proceding data: of user: {}".format(user_id))
            comments = [0 if v is None else v for v in posts]
            #print(receive_posts)
            result = []
            result.extend(comments)
            result.append(user_id)
            result.append(str(start_date))
            #print(result)
            data.append(result)
            #print(data)
        
        df = pd.DataFrame(data, columns = ['comments_freq_avg', 'comments_freq_stddev', 'comments_freq_median', 'comments_freq_q3', 'comments_freq_max', 'comments_freq_min', 'user_id', 'start_date'])
        save_data_to_file("NumOfUsersCommentsFrequency", "feateures_" + str(start_date) + ".csv", df)
        start_date += timedelta(days=15)

In [ ]:
generate_post_activity_time_and_frequency(get_all_ids())

In [ ]:
generate_comments_frequency(get_all_ids())

In [ ]:
# num of words of users comments

##############################

def get_num_of_words_of_comments(user_id, start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT AVG(cs.len), stddev(cs.len),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.len), 
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.len),
                       MAX(cs.len), MIN(cs.len) FROM
                       (SELECT array_length(regexp_split_to_array(c.content, \'\s\'),1) as len FROM {} c 
                       JOIN {} p ON p.id = c.post_id
                       WHERE c.author_id = {} and p.author_id <> {} AND
                       c.date between '{}' and '{}') as cs            
    """.format(comments, posts, user_id, user_id, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()[0]
    conn.commit()
    cur.close()
    features = {
            'written_comments_word_count_avg': float(result[0]) if result[0] is not None else 0.0,
            'written_comments_word_count_stddev': float(result[1]) if result[1] is not None else 0.0,
            'written_comments_word_count_median': float(result[2]) if result[2] is not None else 0.0,
            'written_comments_word_count_q3': float(result[3]) if result[3] is not None else 0.0,
            'written_comments_word_count_max': float(result[4]) if result[4] is not None else 0.0,
            'written_comments_word_count_min': float(result[5]) if result[5] is not None else 0.0
        }
    return features

In [ ]:
def generate_num_of_words_of_comments(users):
    start_date = start_date_comments
    while start_date < end_date_comments:
        print(start_date)
        data = []
        for user_id in users:
            comments = get_num_of_words_of_comments(user_id, start_date, start_date + timedelta(days=30))
            #print("Proceding data: of user: {}".format(user_id))
            comments = [0 if v is None else v for v in comments]
            #print(receive_posts)
            result = []
            result.extend(comments)
            result.append(user_id)
            result.append(str(start_date))
            data.append(result)
        
        df = pd.DataFrame(data, columns = ['written_comments_word_count_avg', 'written_comments_word_count_stddev', 'written_comments_word_count_median', 'written_comments_word_count_q3', 'written_comments_word_count_max', 'written_comments_word_count_min', 'user_id', 'start_date'])
        save_data_to_file("NumOfWordsOFUsersComments", "feateures_" + str(start_date) + ".csv", df)
        start_date += timedelta(days=15)

In [ ]:
# num of words of users posts



#######################
def get_num_of_words_of_posts(user_id, start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT AVG(array_length(regexp_split_to_array(content, \'\s\'),1)),
                       stddev(array_length(regexp_split_to_array(content, \'\s\'),1)),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY array_length(regexp_split_to_array(content, \'\s\'),1)),
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY array_length(regexp_split_to_array(content, \'\s\'),1)),
                       MAX(array_length(regexp_split_to_array(content, \'\s\'),1)),
                       MIN(array_length(regexp_split_to_array(content, \'\s\'),1))
                       FROM {} WHERE author_id = {} AND date between '{}' and '{}'      
    """.format(posts, user_id, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()[0]
    conn.commit()
    cur.close()
    features = {
            'posts_word_count_avg': float(result[0]) if result[0] is not None else 0.0,
            'posts_word_count_stddev': float(result[1]) if result[1] is not None else 0.0,
            'posts_word_count_median': float(result[2]) if result[2] is not None else 0.0,
            'posts_word_count_q3': float(result[3]) if result[3] is not None else 0.0,
            'posts_word_count_max': float(result[4]) if result[4] is not None else 0.0,
            'posts_word_count_min': float(result[5]) if result[5] is not None else 0.0
        }
    return features

In [ ]:
def generate_num_of_words_of_posts(users):
    start_date = start_date_comments
    while start_date < end_date_comments:
        print(start_date)
        data = []
        for user_id in users:
            comments = get_num_of_words_of_posts(user_id, start_date, start_date + timedelta(days=30)).values()
            #print("Proceding data: of user: {}".format(user_id))
            comments = [0 if v is None else v for v in comments]
            #print(receive_posts)
            result = []
            result.extend(comments)
            result.append(user_id)
            result.append(str(start_date))
            data.append(result)
        
        df = pd.DataFrame(data, columns = ['posts_word_count_avg', 'posts_word_count_stddev', 'posts_word_count_median', 'posts_word_count_q3', 'posts_word_count_max', 'posts_word_count_min', 'user_id', 'start_date'])
        save_data_to_file("NumOfWordsOFUsersPosts", "feateures_" + str(start_date) + ".csv", df)
        start_date += timedelta(days=15)

In [ ]:
# num of words in responses for users posts

####################

def get_num_of_words_of_received_post_responses(user_id, start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT AVG(cs.len), stddev(cs.len),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.len),
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.len),
                       MAX(cs.len), MIN(cs.len) FROM
                       (SELECT array_length(regexp_split_to_array(c.content, \'\s\'),1) as len FROM {} c
                       JOIN {} p ON p.id = c.post_id
                       WHERE p.author_id = {} and c.author_id <> {} 
                       AND c.date between '{}' and '{}') as cs
    """.format(comments, posts, user_id, user_id, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()[0]
    conn.commit()
    cur.close()
    features = {
            'received_posts_responses_word_count_avg': float(result[0]) if result[0] is not None else 0.0,
            'received_posts_responses_word_count_stddev': float(result[1]) if result[1] is not None else 0.0,
            'received_posts_responses_word_count_median': float(result[2]) if result[2] is not None else 0.0,
            'received_posts_responses_word_count_q3': float(result[3]) if result[3] is not None else 0.0,
            'received_posts_responses_word_count_max': float(result[4]) if result[4] is not None else 0.0,
            'received_posts_responses_word_count_min': float(result[5]) if result[5] is not None else 0.0
        }
    return features

In [ ]:
def generate_num_of_words_of_received_post_responses(users):
    start_date = start_date_comments
    while start_date < end_date_comments:
        print(start_date)
        data = []
        for user_id in users:
            comments = get_num_of_words_of_received_post_responses(user_id, start_date, start_date + timedelta(days=30)).values()
            #print("Proceding data: of user: {}".format(user_id))
            comments = [0 if v is None else v for v in comments]
            #print(receive_posts)
            result = []
            result.extend(comments)
            result.append(user_id)
            result.append(str(start_date))
            data.append(result)
        
        df = pd.DataFrame(data, columns = ['received_posts_responses_word_count_avg', 'received_posts_responses_word_count_stddev', 'received_posts_responses_word_count_median', 'received_posts_responses_word_count_q3', 'received_posts_responses_word_count_max', 'received_posts_responses_word_count_min', 'user_id', 'start_date'])
        save_data_to_file("NumOfWordsOFResponsesToUsersPosts", "feateures_" + str(start_date) + ".csv", df)
        start_date += timedelta(days=15)

In [ ]:
# num of words in responses for users posts

def get_num_of_words_of_own_post_responses(user_id, start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT AVG(cs.len), stddev(cs.len),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.len),
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.len),
                       MAX(cs.len), MIN(cs.len) FROM
                       (SELECT array_length(regexp_split_to_array(c.content, \'\s\'),1) as len FROM {} c
                       JOIN {} p ON p.id = c.post_id
                       WHERE p.author_id = {} and c.author_id = {} AND
                       c.date between '{}' and '{}') as cs
    """.format(comments, posts, user_id, user_id, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()[0]
    conn.commit()
    cur.close()
    features = {
            'own_posts_responses_word_count_avg': float(result[0]) if result[0] is not None else 0.0,
            'own_posts_responses_word_count_stddev': float(result[1]) if result[1] is not None else 0.0,
            'own_posts_responses_word_count_median': float(result[2]) if result[2] is not None else 0.0,
            'own_posts_responses_word_count_q3': float(result[3]) if result[3] is not None else 0.0,
            'own_posts_responses_word_count_max': float(result[4]) if result[4] is not None else 0.0,
            'own_posts_responses_word_count_min': float(result[5]) if result[5] is not None else 0.0
        }
    return features

In [ ]:
def generate_num_of_words_of_own_post_responses(users):
    start_date = start_date_comments
    while start_date < end_date_comments:
        print(start_date)
        data = []
        for user_id in users:
            comments = get_num_of_words_of_own_post_responses(user_id, start_date, start_date + timedelta(days=30)).values()
            #print("Proceding data: of user: {}".format(user_id))
            comments = [0 if v is None else v for v in comments]
            #print(receive_posts)
            result = []
            result.extend(comments)
            result.append(user_id)
            result.append(str(start_date))
            data.append(result)
        
        df = pd.DataFrame(data, columns = ['own_posts_responses_word_count_avg', 'own_posts_responses_word_count_stddev', 'own_posts_responses_word_count_median', 'own_posts_responses_word_count_q3', 'own_posts_responses_word_count_max', 'own_posts_responses_word_count_min', 'user_id', 'start_date'])
        save_data_to_file("NumOfWordsOf_OWN_ResponsesToUsersPosts", "feateures_" + str(start_date) + ".csv", df)
        start_date += timedelta(days=15)

In [ ]:
# 2008-12-09
# 2008-12-24
# 2009-01-08
# 2009-01-23
# 2009-02-07
# 2009-02-22

In [ ]:
generate_number_of_written_comments_to_other_users(get_all_ids())

In [ ]:
generate_number_of_received_responses_under_users_comments(get_all_ids())

In [ ]:
generate_number_of_comments_written_by_user_under_his_own_posts(get_all_ids())

In [ ]:
generate_number_of_users_posts(get_all_ids())

In [ ]:
generate_post_activity_time_and_frequency(get_all_ids())

In [ ]:
generate_comments_frequency(get_all_ids())

In [ ]:
generate_num_of_words_of_comments(get_all_ids())

In [ ]:
generate_num_of_words_of_posts(get_all_ids())

In [ ]:
generate_num_of_wor ds_of_own_post_responses(get_all_ids())

In [ ]:
generate_num_of_words_of_received_post_responses(get_all_ids())

In [ ]:
generate_number_of_received_responses_for_users_posts(get_all_ids()[200000:400000])

In [ ]:
conn.close()

###                      NOWY KOD

In [ ]:
import psycopg2
import pandas as pd
import os
import itertools
import numpy as np

conn = psycopg2.connect(
    host='localhost',
    port=54320,
    dbname='my_database',
    password='pass',
    user='user',
    )
conn.autocommit = False


In [ ]:
import datetime
from datetime import timedelta 

authors = "authors"
comments = "comments"
posts = "posts"
category = "category"
topics = "topics"
post_sentiment = "post_sentiment"
comment_sentiment = "comment_sentiment"
user_id = 590

start_date_comments = datetime.date(2008, 12, 9)
end_date_comments = datetime.date(2013,11, 16)
# start_date_comments = datetime.date(2011, 12, 24)
# end_date_comments = datetime.date(2012, 2, 23)
# 2008-12-09 18:41:00 - comments 
# 2013-11-16 21:35:00 - comments

# 2005-12-09 09:50:17 - posts
# 2013-11-14 10:13:00 - posts

In [ ]:
df = pd.read_csv("FeatureTableCSV.csv", sep=";")

In [ ]:
def get_all_users():
    cur = conn.cursor()
    query = """
    SELECT id, name FROM {} order by id
    """.format(authors)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
def get_all_ids():
    result = get_all_users()
    return [x[0] for x in result]

In [ ]:
def save_data_to_file(folder_name, file_name, data):
    try:
        # Create target Directory
        os.mkdir(folder_name)
        print("Directory " , folder_name ,  " Created ") 
    except FileExistsError:
        e = 1
        #print("Directory " , folder_name ,  " already exists")
    data.to_csv(folder_name + "/" + file_name, index=False)

In [ ]:
def clear_none_values(results):
    data = []
    for res in results:
        res = [0 if v is None else v for v in res]
        data.append(res)
    return data

In [ ]:
start_date = datetime.date(2011, 12, 24)
end_date = datetime.date(2012, 2, 23)

In [ ]:
def generate_feature(start_date, end_date, is_clear, feature_function, columns, folder_name):
    
    while start_date < end_date:
        print(start_date)
        results = feature_function(start_date, start_date + timedelta(days=28))
        res = []
        if is_clear:
            res = results
        else:
            res = clear_none_values(results)
        data = []
        data.extend(res)
        df = pd.DataFrame(data, columns = columns )
        df['start_date'] = pd.Series(start_date, index=df.index)
        df['end_date'] = pd.Series(end_date, index=df.index)
        save_data_to_file(folder_name, "feature_" + str(start_date) + ".csv", df)
        start_date += timedelta(days=14)

In [ ]:
def get_number_of_users_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT author_id, COUNT(*) FROM {}
    WHERE date between '{}' and '{}'
    GROUP BY author_id
    ORDER BY author_id
    """.format(posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
#"NumberOfUsersPostsWithoutZeros"
folder = "NumberOfUserPosts"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, True, get_number_of_users_posts, columns, folder)

In [ ]:
def get_number_of_written_comments_to_other_users(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT c.author_id, COUNT(*) FROM {} c
    JOIN {} p ON p.id = c.post_id AND c.author_id <> p.author_id
    WHERE c.date between '{}' and '{}'
    GROUP BY c.author_id
    ORDER BY c.author_id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
folder = "NumberOfCommentsToOtherUsers"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, True, get_number_of_written_comments_to_other_users, columns, folder)

In [ ]:
def get_number_of_received_responses_to_users_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT cs.id, AVG(cs.resp_amount), stddev(cs.resp_amount),
    percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.resp_amount),
    percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.resp_amount),
    MAX(cs.resp_amount), MIN(cs.resp_amount) FROM
    (SELECT p.author_id as id, COUNT(*) as resp_amount FROM {} c
    JOIN {} p ON p.id = c.post_id AND p.author_id <> c.author_id
    WHERE c.date between '{}' and '{}'
    GROUP BY p.id, p.author_id) as cs
    GROUP BY cs.id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
folder = "NumberOfReceivedResponsesToPost"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_number_of_received_responses_to_users_posts, columns, folder)

In [ ]:
def get_number_of_received_unique_users_responses_to_users_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT cs.id, AVG(cs.resp_amount), stddev(cs.resp_amount),
    percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.resp_amount),
    percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.resp_amount),
    MAX(cs.resp_amount), MIN(cs.resp_amount) FROM
    (SELECT p.author_id as id, COUNT(c.author_id ) as resp_amount 
    FROM {} c JOIN {} p ON p.id = c.post_id 
    AND p.author_id <> c.author_id 
    WHERE c.date between '{}' AND '{}' 
    GROUP BY p.id, p.author_id) as cs
    GROUP BY cs.id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
folder = "NumberOfReceivedUniqueUsersResponsesToPost"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_number_of_received_unique_users_responses_to_users_posts, columns, folder)

In [ ]:
def get_number_of_all_responses_from_unique_users_in_slot(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT p.author_id as id, COUNT(c.author_id ) as resp_amount 
    FROM {} c JOIN {} p ON p.id = c.post_id 
    AND p.author_id <> c.author_id 
    WHERE c.date between '{}' AND '{}' 
    GROUP BY p.author_id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
folder = "NumberOfAllResponsesFromUniqueUsersInSlot"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_number_of_all_responses_from_unique_users_in_slot, columns, folder)

In [ ]:
def get_number_of_comments_written_by_user_under_his_own_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT cs.id, AVG(cs.resp_amount), stddev(cs.resp_amount),
    percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.resp_amount),
    percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.resp_amount),
    MAX(cs.resp_amount), MIN(cs.resp_amount) FROM
    (SELECT p.author_id as id, COUNT(*) as resp_amount FROM {} c
    JOIN {} p ON p.id = c.post_id AND p.author_id = c.author_id
    WHERE c.date between '{}' and '{}'
    GROUP BY p.id, p.author_id) as cs
    GROUP BY cs.id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
folder = "NumberOfOwnResponsesToPost"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_number_of_comments_written_by_user_under_his_own_posts, columns, folder)

In [ ]:
def get_number_of_received_responses_under_users_comments(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT aggregate.id, AVG(aggregate.resp_amount), stddev(aggregate.resp_amount),
    percentile_cont(0.5) WITHIN GROUP (ORDER BY aggregate.resp_amount),
    percentile_cont(0.75) WITHIN GROUP (ORDER BY aggregate.resp_amount),
    MAX(aggregate.resp_amount), MIN(aggregate.resp_amount) FROM
    (SELECT parent_comments.author_id as id, COUNT(*) as resp_amount FROM {} parent_comments
    JOIN {} children_comments ON parent_comments.id = children_comments.parentcomment_id AND parent_comments.author_id <> children_comments.author_id
    WHERE children_comments.date between '{}' and '{}'
    GROUP BY parent_comments.id, parent_comments.author_id) as aggregate
    GROUP BY aggregate.id
    """.format(comments, comments, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"])

In [ ]:
folder = "NumberOfReceivedResponsesUnderComments"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_number_of_received_responses_under_users_comments, columns, folder)

In [ ]:
def get_num_of_words_of_comments(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT cs.id, AVG(cs.len), stddev(cs.len),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.len), 
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.len),
                       MAX(cs.len), MIN(cs.len) FROM
                       (SELECT c.author_id as id, array_length(regexp_split_to_array(c.content, \'\s\'),1) as len FROM {} c 
                       JOIN {} p ON p.id = c.post_id and c.author_id <> p.author_id
                       WHERE c.date between '{}' and '{}') as cs
                       GROUP BY cs.id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
folder = "NumberOfWordsInUserComments"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_num_of_words_of_comments, columns, folder)

In [ ]:
def get_num_of_words_of_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT author_id, AVG(array_length(regexp_split_to_array(content, \'\s\'),1)),
                       stddev(array_length(regexp_split_to_array(content, \'\s\'),1)),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY array_length(regexp_split_to_array(content, \'\s\'),1)),
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY array_length(regexp_split_to_array(content, \'\s\'),1)),
                       MAX(array_length(regexp_split_to_array(content, \'\s\'),1)),
                       MIN(array_length(regexp_split_to_array(content, \'\s\'),1))
                       FROM {} WHERE date between '{}' and '{}'
                       GROUP BY author_id
    """.format(posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
folder = "NumberOfWordsInUserPosts"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_num_of_words_of_posts, columns, folder)

In [ ]:
def get_num_of_words_of_received_post_responses(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT cs.id, AVG(cs.len), stddev(cs.len),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.len),
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.len),
                       MAX(cs.len), MIN(cs.len) FROM
                       (SELECT p.author_id as id, array_length(regexp_split_to_array(c.content, \'\s\'),1) as len FROM {} c
                       JOIN {} p ON p.id = c.post_id AND p.author_id <> c.author_id
                       WHERE c.date between '{}' and '{}') as cs
                       GROUP BY cs.id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
folder = "NumberOfWordsInResponsesToUserPosts"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_num_of_words_of_received_post_responses, columns, folder)

In [ ]:
def get_num_of_words_of_own_post_responses(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT cs.id, AVG(cs.len), stddev(cs.len),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.len),
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.len),
                       MAX(cs.len), MIN(cs.len) FROM
                       (SELECT p.author_id as id, array_length(regexp_split_to_array(c.content, \'\s\'),1) as len FROM {} c
                       JOIN {} p ON p.id = c.post_id AND p.author_id = c.author_id
                       WHERE c.date between '{}' and '{}') as cs
                       GROUP BY cs.id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
folder = "NumberOfWordsInOwnPostResponses"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_num_of_words_of_own_post_responses, columns, folder)

In [ ]:
def get_user_posts_sentiment_info(start_date, end_date):
    cur = conn.cursor()
    query = """SELECT p.author_id, AVG(ps.compound), stddev(ps.compound),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY ps.compound),
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY ps.compound),
                       MAX(ps.compound), MIN(ps.compound)
                       FROM {} ps JOIN {} p ON ps.id = p.id 
                       WHERE p.date between '{}' and '{}'
                       GROUP BY p.author_id
    """.format(post_sentiment, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result


In [ ]:
res = get_user_posts_sentiment_info(start_date, end_date)
print(res[:10])

In [ ]:
folder = "PostSentiment"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_user_posts_sentiment_info, columns, folder)

In [ ]:
def get_user_comments_sentiment_info(start_date, end_date):
    cur = conn.cursor()
    query = """SELECT c.author_id, AVG(cs.compound), stddev(cs.compound),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.compound),
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.compound),
                       MAX(cs.compound), MIN(cs.compound)
                       FROM {} cs JOIN {} c ON cs.id = c.id 
                       WHERE c.date between '{}' and '{}'
                       GROUP BY c.author_id
    """.format(comment_sentiment, comments, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
res = get_user_comments_sentiment_info(start_date, end_date)
print(res[:10])

In [ ]:
folder = "CommentSentiment"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_user_comments_sentiment_info, columns, folder)

In [ ]:
def get_user_sentiment_info_received_comments(start_date, end_date):
    cur = conn.cursor()
    query = """SELECT p.author_id, AVG(compound), stddev(compound),
                    percentile_cont(0.5) WITHIN GROUP (ORDER BY compound),
                    percentile_cont(0.75) WITHIN GROUP (ORDER BY compound),
                    MAX(compound), MIN(compound)
                    FROM {} cs
                    JOIN {} c ON c.id = cs.id
                    JOIN {} p ON p.id = c.post_id and c.author_id <> p.author_id
                    WHERE c.date between '{}' and '{}'
                    GROUP BY p.author_id
    """.format(comment_sentiment, comments, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
res = get_user_sentiment_info_received_comments(start_date, end_date)
print(res[:10])

In [ ]:
folder = "ReceivedResponsesSentiment"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_user_sentiment_info_received_comments, columns, folder)

In [ ]:
def q75(x):
    return x.quantile(0.75)

In [ ]:
def generate_frequency_feature(start_date, end_date, feature_frequency_function, columns, final_columns, folder_name):
    
    while start_date < end_date:
        print(start_date)
        temp_end_date = start_date + timedelta(days=28)
        temp_start_date = start_date
        dates = []
        while temp_start_date < temp_end_date:
            results = feature_frequency_function(start_date, temp_end_date, temp_start_date, temp_start_date + timedelta(days=7))
            
            data = []
            data.extend(results)
            df = pd.DataFrame(data, columns = columns )
            df['start_date'] = pd.Series(temp_start_date, index=df.index)
            df['end_date'] = pd.Series(temp_start_date + timedelta(days=7), index=df.index)
            
            dates.append(df)
            temp_start_date += timedelta(days=7)
        res = pd.concat(dates, ignore_index=True)
        if res.size != 0:
            res[columns[1]] = pd.to_numeric(res[columns[1]])
#             quartile = res.groupby(['user_id'])[columns[1]].quantile(0.75)
            stats_df = (res.groupby('user_id')[columns[1]].agg(('mean', 'std','median', q75, 'max', 'min')).reset_index())
#             merged_df = pd.merge(stats_df, quartile, on='user_id')
            final_df = stats_df.fillna(0)
            final_df.columns = final_columns
            save_data_to_file(folder_name, "feature_" + str(start_date) + ".csv", final_df)
        else:
            empty = pd.DataFrame(data, columns = final_columns )
            save_data_to_file(folder_name, "feature_" + str(start_date) + ".csv", empty)
        start_date += timedelta(days=14)

In [ ]:
def get_post_frequency(real_starts_date, real_end_date, start_date, end_date, timedelta_days=28):
    cur = conn.cursor()
    query = """
    WITH all_users_in_period AS (
    SELECT author_id FROM {} WHERE date between '{}' and '{}' 
    ),
    week AS (SELECT author_id as id, COUNT(*) as num FROM {} 
    WHERE date between '{}' and '{}' 
    GROUP BY author_id)
    
    SELECT a.author_id, coalesce(w.num, 0) as num 
    FROM all_users_in_period a
    LEFT JOIN week w ON a.author_id = w.id
    """.format(posts, real_starts_date, real_end_date, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
columns = ["user_id", "number_of_posts"]
folder = "FrequencyOfPosts"
final_columns = ['user_id']
final_columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
# final_columns = ['user_id', 'mean_post_frequency', 'std_post_frequency','median_post_frequency','min_post_frequency', 'max_post_frequency', 'q3_post_frequency']
generate_frequency_feature(start_date_comments, end_date_comments, get_post_frequency, columns, final_columns, folder)

In [ ]:
# frequency of writting comments
# wszyscy uzytkownicy ktorzy commentowali 
def get_comments_frequency(real_starts_date, real_end_date, start_date, end_date, timedelta_days=7):
    cur = conn.cursor()
    query = """
    WITH all_users_in_period AS (
    SELECT author_id FROM {} WHERE date between '{}' and '{}'
    ),
    week AS (SELECT author_id as id, COUNT(*) as num FROM {} 
    WHERE date between '{}' and '{}' 
    GROUP BY author_id)
    
    SELECT a.author_id, coalesce(w.num, 0) as num 
    FROM all_users_in_period a
    LEFT JOIN week w ON a.author_id = w.id
    """.format(comments, real_starts_date, real_end_date,comments, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res


In [ ]:
columns = ["user_id", "number_of_comments"]

folder = "FrequencyOfComments"
final_columns = ['user_id']
final_columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
# final_columns = ['user_id', 'mean_comments_frequency', 'std_comments_frequency','median_comments_frequency','min_comments_frequency', 'max_comments_frequency', 'q3_comments_frequency']
generate_frequency_feature(start_date_comments, end_date_comments, get_comments_frequency, columns, final_columns, folder)

In [ ]:
def get_post_activity_without_zeros(start_date, end_date):
    cur = conn.cursor()
    query = """
    WITH disctinct_months AS (
    SELECT DISTINCT author_id, date::date FROM {} WHERE date between '{}' AND '{}'
    ) 
    SELECT author_id as id, count(*) AS post_activity 
    FROM disctinct_months GROUP BY author_id
    """.format(posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
folder = "PostActivityTime"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_post_activity_without_zeros, columns, folder)

In [ ]:
cur = conn.cursor()
query = """
SELECT id, name FROM authors
""".format(posts, start_date, end_date)
cur.execute(query)
res = cur.fetchall()
conn.commit()
cur.close()
columns = ["id","name"]
data = []
data.extend(res)
df = pd.DataFrame(data, columns = columns )
save_data_to_file("Authors", "authors.csv", df)

### with zeros

In [ ]:
def get_number_of_users_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT id, up.num FROM {} a LEFT JOIN (SELECT author_id, COUNT(*) as num FROM {}
    WHERE date between '{}' and '{}'
    GROUP BY author_id
    ORDER BY author_id) up ON up.author_id = a.id
    """.format(authors, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
#"NumberOfUsersPostsWithoutZeros"
columns = ['user_id', 'number_of_posts']
generate_feature(start_date_comments, end_date_comments, False, get_number_of_users_posts, columns, "NumberOfUsersPosts")

In [ ]:
def get_number_of_written_comments_to_other_users(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT a.id, q.num FROM {} a LEFT JOIN 
    (SELECT c.author_id, COUNT(*) as num FROM {} c
    JOIN {} p ON p.id = c.post_id AND c.author_id <> p.author_id
    WHERE c.date between '{}' and '{}'
    GROUP BY c.author_id
    ORDER BY c.author_id) q ON q.author_id = a.id
    """.format(authors, comments, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
columns = ['user_id', 'number_of_written_comments_to_other_users_posts']
generate_feature(start_date_comments, end_date_comments, False, get_number_of_written_comments_to_other_users, columns, "NumberOfWrittenCommentsToOtherUsersPosts")

In [ ]:
def get_number_of_received_responses_to_users_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT a.id, q.avg, q.std, q.median, q.q3, q.max, q.min FROM {} a LEFT JOIN 
    (SELECT cs.id, AVG(cs.resp_amount) as avg, stddev(cs.resp_amount) as std,
    percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.resp_amount) as median,
    percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.resp_amount) as q3,
    MAX(cs.resp_amount) as max, MIN(cs.resp_amount) as min FROM
    (SELECT p.author_id as id, COUNT(*) as resp_amount FROM {} c
    JOIN {} p ON p.id = c.post_id AND p.author_id <> c.author_id
    WHERE c.date between '{}' and '{}'
    GROUP BY p.id, p.author_id) as cs
    GROUP BY cs.id
    ) q ON q.id = a.id
    """.format(authors, comments, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
columns = ['user_id', 'number_of_received_responses_to_users_posts_avg', 'number_of_received_responses_to_users_posts_std', 'number_of_received_responses_to_users_posts_median', 'number_of_received_responses_to_users_posts_q3', 'number_of_received_responses_to_users_posts_max', 'number_of_received_responses_to_users_posts_min']
generate_feature(start_date_comments, end_date_comments, False, get_number_of_received_responses_to_users_posts, columns, "NumberOfReceivedResponsesToUsersPosts")

In [ ]:
def get_number_of_comments_written_by_user_under_his_own_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT a.id, q.avg, q.std, q.median, q.q3, q.max, q.min FROM {} a LEFT JOIN (
    SELECT cs.id, AVG(cs.resp_amount) as avg, stddev(cs.resp_amount) as std,
    percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.resp_amount) as median,
    percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.resp_amount) as q3,
    MAX(cs.resp_amount) as max, MIN(cs.resp_amount) as min FROM
    (SELECT p.author_id as id, COUNT(*) as resp_amount FROM {} c
    JOIN {} p ON p.id = c.post_id AND p.author_id = c.author_id
    WHERE c.date between '{}' and '{}'
    GROUP BY p.id, p.author_id) as cs
    GROUP BY cs.id
    ) q ON q.id = a.id
    """.format(authors, comments, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
columns = ['user_id', 'number_of_comments_written_by_user_under_his_own_posts_avg', 'number_of_comments_written_by_user_under_his_own_posts_std', 'number_of_comments_written_by_user_under_his_own_posts_median', 'number_of_comments_written_by_user_under_his_own_posts_q3', 'number_of_comments_written_by_user_under_his_own_posts_max', 'number_of_comments_written_by_user_under_his_own_posts_min']
generate_feature(start_date_comments, end_date_comments, False, get_number_of_comments_written_by_user_under_his_own_posts, columns, "NumberOfCommentsWrittenByUserUnderHisOwnPosts")

In [ ]:
def get_number_of_received_responses_under_users_comments(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT a.id, q.avg, q.std, q.median, q.q3, q.max, q.min FROM {} a LEFT JOIN (
    SELECT cs.id, AVG(cs.resp_amount) as avg, stddev(cs.resp_amount) as std,
    percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.resp_amount) as median,
    percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.resp_amount) as q3,
    MAX(cs.resp_amount) as max, MIN(cs.resp_amount) as min FROM
    (SELECT c.author_id as id, COUNT(*) as resp_amount FROM {} c
    JOIN {} c2 ON c.parentcomment_id = c2.id AND c2.author_id <> c.author_id
    WHERE c.date between '{}' and '{}'
    GROUP BY c.parentcomment_id, c.author_id) as cs
    GROUP BY cs.id
    ) q ON q.id = a.id
    """.format(authors, comments, comments, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
columns = ['user_id', 'number_of_received_responses_under_users_comments_avg', 'number_of_received_responses_under_users_comments_std', 'number_of_received_responses_under_users_comments_median', 'number_of_received_responses_under_users_comments_q3', 'number_of_received_responses_under_users_comments_max', 'number_of_received_responses_under_users_comments_min']
generate_feature(start_date_comments, end_date_comments, False, get_number_of_received_responses_under_users_comments, columns, "NumberOfReceivedResponsesUnderUsersComments")

In [ ]:
def get_num_of_words_of_comments(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT a.id, q.avg, q.std, q.median, q.q3, q.max, q.min FROM {} a LEFT JOIN (
    SELECT cs.id, AVG(cs.len) as avg, stddev(cs.len) as std,
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.len) as median, 
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.len) as q3,
                       MAX(cs.len) as max, MIN(cs.len) as min FROM
                       (SELECT c.author_id as id, array_length(regexp_split_to_array(c.content, \'\s\'),1) as len FROM {} c 
                       JOIN {} p ON p.id = c.post_id and c.author_id <> p.author_id
                       WHERE c.date between '{}' and '{}') as cs
                       GROUP BY cs.id
                        ) q ON q.id = a.id
    """.format(authors, comments, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
columns = ['user_id', 'number_of_words_in_users_comments_avg', 'number_of_words_in_users_comments_std', 'number_of_words_in_users_comments_median', 'number_of_words_in_users_comments_q3', 'number_of_words_in_users_comments_max', 'number_of_words_in_users_comments_min']
generate_feature(start_date_comments, end_date_comments, False, get_num_of_words_of_comments, columns, "NumberOfWordsInUsersComments")

In [ ]:
def get_num_of_words_of_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT a.id, q.avg, q.std, q.median, q.q3, q.max, q.min FROM {} a LEFT JOIN (
    SELECT author_id as id, AVG(array_length(regexp_split_to_array(content, \'\s\'),1)) as avg,
                       stddev(array_length(regexp_split_to_array(content, \'\s\'),1)) as std,
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY array_length(regexp_split_to_array(content, \'\s\'),1)) as median,
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY array_length(regexp_split_to_array(content, \'\s\'),1)) as q3,
                       MAX(array_length(regexp_split_to_array(content, \'\s\'),1)) as max,
                       MIN(array_length(regexp_split_to_array(content, \'\s\'),1)) as min
                       FROM {} WHERE date between '{}' and '{}'
                       GROUP BY author_id
                       ) q ON q.id = a.id
    """.format(authors, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
columns = ['user_id', 'number_of_words_in_users_posts_avg', 'number_of_words_in_users_posts_std', 'number_of_words_in_users_posts_median', 'number_of_words_in_users_posts_q3', 'number_of_words_in_users_posts_max', 'number_of_words_in_users_posts_min']
generate_feature(start_date_comments, end_date_comments, False, get_num_of_words_of_posts, columns, "NumberOfWordsInUsersPosts")

In [ ]:
def get_num_of_words_of_received_post_responses(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT a.id, q.avg, q.std, q.median, q.q3, q.max, q.min FROM {} a LEFT JOIN (
    SELECT cs.id, AVG(cs.len) as avg, stddev(cs.len) as std,
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.len) as median,
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.len) as q3,
                       MAX(cs.len) as max, MIN(cs.len) as min FROM
                       (SELECT p.author_id as id, array_length(regexp_split_to_array(c.content, \'\s\'),1) as len FROM {} c
                       JOIN {} p ON p.id = c.post_id AND p.author_id <> c.author_id
                       WHERE c.date between '{}' and '{}') as cs
                       GROUP BY cs.id
                       ) q ON q.id = a.id
    """.format(authors, comments, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
columns = ['user_id', 'number_of_words_in_responses_of_users_posts_avg', 'number_of_words_in_responses_of_users_posts_std', 'nnumber_of_words_in_responses_of_users_posts_median', 'number_of_words_in_responses_of_users_posts_q3', 'number_of_words_in_responses_of_users_posts_max', 'number_of_words_in_responses_of_users_posts_min']
generate_feature(start_date_comments, end_date_comments, False, get_num_of_words_of_received_post_responses, columns, "NumberOfWordsInResponsesOfUsersPosts")

In [ ]:
def get_num_of_words_of_own_post_responses(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT a.id, q.avg, q.std, q.median, q.q3, q.max, q.min FROM {} a LEFT JOIN (
    SELECT cs.id, AVG(cs.len) as avg, stddev(cs.len) as std,
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.len) as median,
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.len) as q3,
                       MAX(cs.len) as max, MIN(cs.len) as min FROM
                       (SELECT p.author_id as id, array_length(regexp_split_to_array(c.content, \'\s\'),1) as len FROM {} c
                       JOIN {} p ON p.id = c.post_id AND p.author_id = c.author_id
                       WHERE c.date between '{}' and '{}') as cs
                       GROUP BY cs.id
                       ) q ON q.id = a.id
    """.format(authors, comments, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
columns = ['user_id', 'number_of_words_in_own_responses_of_users_posts_avg', 'number_of_words_in_own_responses_of_users_posts_std', 'number_of_words_in_own_responses_of_users_posts_median', 'number_of_words_in_own_responses_of_users_posts_q3', 'number_of_words_in_own_responses_of_users_posts_max', 'number_of_words_in_own_responses_of_users_posts_min']
generate_feature(start_date_comments, end_date_comments, False, get_num_of_words_of_own_post_responses, columns, "NumberOfWordsInOwnResponsesOfUsersPosts")

In [ ]:
def get_user_posts_sentiment_info(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT a.id, q.avg, q.std, q.median, q.q3, q.max, q.min FROM {} a LEFT JOIN (
    SELECT p.author_id as id, AVG(ps.compound) as avg, stddev(ps.compound) as std,
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY ps.compound) as median,
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY ps.compound) as q3,
                       MAX(ps.compound) as max, MIN(ps.compound) as min
                       FROM {} ps JOIN {} p ON ps.id = p.id 
                       WHERE p.date between '{}' and '{}'
                       GROUP BY p.author_id
                       ) q ON q.id = a.id
    """.format(authors, post_sentiment, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
columns = ['user_id', 'posts_sentiment_avg', 'posts_sentiment_stddev', 'posts_sentiment_median', 'posts_sentiment_q3', 'posts_sentiment_max', 'posts_sentiment_min']
generate_feature(start_date_comments, end_date_comments, False, get_user_posts_sentiment_info, columns, "SentimentOfUsersPosts")

In [ ]:
def get_user_comments_sentiment_info(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT a.id, q.avg, q.std, q.median, q.q3, q.max, q.min FROM {} a LEFT JOIN (
    SELECT c.author_id as id, AVG(cs.compound) as avg, stddev(cs.compound) as std,
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.compound) as median,
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.compound) as q3,
                       MAX(cs.compound) as max, MIN(cs.compound) as min
                       FROM {} cs JOIN {} c ON cs.id = c.id 
                       WHERE c.date between '{}' and '{}'
                       GROUP BY c.author_id
                       ) q ON q.id = a.id
    """.format(authors, comment_sentiment, comments, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
columns = ['user_id', 'comments_sentiment_avg', 'comments_sentiment_stddev', 'comments_sentiment_median', 'comments_sentiment_q3', 'comments_sentiment_max', 'comments_sentiment_min']
generate_feature(start_date_comments, end_date_comments, False, get_user_comments_sentiment_info, columns, "SentimentOfUsersComments")

In [ ]:
def get_user_sentiment_info_received_comments(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT a.id, q.avg, q.std, q.median, q.q3, q.max, q.min FROM {} a LEFT JOIN (
    SELECT p.author_id as id, AVG(compound) as avg, stddev(compound) as std,
                    percentile_cont(0.5) WITHIN GROUP (ORDER BY compound) as median,
                    percentile_cont(0.75) WITHIN GROUP (ORDER BY compound) as q3,
                    MAX(compound) as max, MIN(compound) as min
                    FROM {} cs
                    JOIN {} c ON c.id = cs.id
                    JOIN {} p ON p.id = c.post_id and c.author_id <> p.author_id
                    WHERE c.date between '{}' and '{}'
                    GROUP BY p.author_id
                    ) q ON q.id = a.id
    """.format(authors, comment_sentiment, comments, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
columns = ['user_id', 'received_comments_sentiment_avg', 'received_comments_sentiment_stddev', 'received_comments_sentiment__median', 'received_comments_sentiment_q3', 'received_comments_sentiment_max', 'received_comments_sentiment_min']
generate_feature(start_date_comments, end_date_comments, False, get_user_sentiment_info_received_comments, columns, "SentimentOfUsersResponsesToUserPosts")

In [ ]:
def generate_frequency_feature(start_date, end_date, feature_frequency_function, columns, final_columns, folder_name):
    
    while start_date < end_date:
        print(start_date)
        temp_end_date = start_date + timedelta(days=28)
        temp_start_date = start_date
        dates = []
        while temp_start_date < temp_end_date:
            results = feature_frequency_function(temp_start_date, temp_start_date + timedelta(days=7))
            results = clear_none_values(results)
            data = []
            data.extend(results)
            df = pd.DataFrame(data, columns = columns )
            df['start_date'] = pd.Series(temp_start_date, index=df.index)
            df['end_date'] = pd.Series(temp_start_date + timedelta(days=7), index=df.index)
            
            dates.append(df)
            temp_start_date += timedelta(days=7)
        res = pd.concat(dates, ignore_index=True)
        if res.size != 0:
            res[columns[1]] = pd.to_numeric(res[columns[1]])
            quartile = res.groupby(['user_id'])[columns[1]].quantile(0.75)
            stats_df = (res.groupby('user_id')[columns[1]].agg(('mean', 'std','median', 'min', 'max')).reset_index())
            merged_df = pd.merge(stats_df, quartile, on='user_id')
            final_df = merged_df.fillna(0)
            final_df.columns = final_columns
            save_data_to_file(folder_name, "feature_" + str(start_date) + ".csv", final_df)
        else:
            empty = pd.DataFrame(data, columns = final_columns )
            save_data_to_file(folder_name, "feature_" + str(start_date) + ".csv", empty)
        start_date += timedelta(days=14)

In [ ]:
def get_post_activity_time_and_frequency(start_date, end_date, timedelta_days=7):
    cur = conn.cursor()
    query = """
    SELECT a.id, q.num FROM {} a LEFT JOIN (
    SELECT author_id as id, COUNT(*) as num FROM {} 
                    WHERE date between '{}' and '{}' 
                    GROUP BY author_id
                    ) q ON q.id = a.id
    """.format(authors, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
columns = ["user_id", "number_of_posts"]
final_columns = ['user_id', 'mean_post_frequency', 'std_post_frequency','median_post_frequency','min_post_frequency', 'max_post_frequency', 'q3_post_frequency']
generate_frequency_feature(start_date_comments, end_date_comments, get_post_activity_time_and_frequency, columns, final_columns, "FrequencyOfUserPosts")

In [ ]:
# frequency of writting comments
def get_comments_frequency(start_date, end_date, timedelta_days=7):
    cur = conn.cursor()
    query = """
    SELECT a.id, q.num FROM {} a LEFT JOIN (
    SELECT author_id as id, COUNT(*) as num FROM {} 
                    WHERE date between '{}' and '{}' 
                    GROUP BY author_id
                    ) q ON q.id = a.id
    """.format(authors, comments, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res


In [ ]:
columns = ["user_id", "number_of_comments"]
final_columns = ['user_id', 'mean_comments_frequency', 'std_comments_frequency','median_comments_frequency','min_comments_frequency', 'max_comments_frequency', 'q3_comments_frequency']
generate_frequency_feature(start_date_comments, end_date_comments, get_comments_frequency, columns, final_columns, "FrequencyOfUserComments")

In [ ]:
conn.close()